In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time
import collections
import math
import random
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
import csv

In [2]:
dir_name = 'datasets'
file_name = 'select_description_from_sys_district.csv'
if not os.path.exists(dir_name+os.sep+file_name):
    file_name = urlretrieve(url + file_name, dir_name+os.sep+file_name)
else:
    print('文件 ',file_name, '已经存在.')

num_files = 1
filenames = [file_name]
def read_data(filename):
  
  with open(filename) as f:
    data = tf.compat.as_str(f.read())
    # 将所有文本均设为小写形式
    data = data.lower()
    data = list(data)
  return data

global documents
documents = []
for i in range(num_files):    
    print('\n处理文件 %s'%os.path.join(dir_name,filenames[i]))
    chars = read_data(os.path.join(dir_name,filenames[i]))
    
    # 将文本分解成bigram形式
    two_grams = [''.join(chars[ch_i:ch_i+2]) for ch_i in range(0,len(chars)-2,2)]
    # 创建一个带有bigram形式的列表
    documents.append(two_grams)
    print('数据大小 (字符) (文档 %d) %d' %(i,len(two_grams)))
    print('样本字符串 (文档 %d) %s'%(i,two_grams[:50]))

文件  select_description_from_sys_district.csv 已经存在.

处理文件 datasets/select_description_from_sys_district.csv
数据大小 (字符) (文档 0) 447415
样本字符串 (文档 0) ['20', '19', '年开', '业6', '0间', '房。', '上海', '时生', '隅园', '林文', '化酒', '店：', '20', '17', '年的', '秋末', '，时', '生隅', '初见', '。彼', '时的', '心愿', '，想', '要在', '快节', '奏的', '都市', '，寻', '一处', '“讲', '诉时', '光和', '生命', '的角', '落”', '。不', '问对', '错，', '不闻', '浮沉', '。这', '便是', '“时', '生隅', '”的', '由来', '。 ', '这一', '次，', '我们']


In [3]:
text2 = open(dir_name+os.sep+file_name, 'rb').read().decode(encoding='utf-8')

# 文本长度是指文本中的字符个数
print ('Length of text: {} characters'.format(len(text2)))

Length of text: 894832 characters


In [4]:
print(text2[:255])

2019年开业60间房。上海时生隅园林文化酒店：2017年的秋末，时生隅初见。彼时的心愿，想要在快节奏的都市，寻一处“讲诉时光和生命的角落”。不问对错，不闻浮沉。这便是“时生隅”的由来。 这一次，我们秉承初心，在闹中取静的川沙地铁站商圈，觅得一方天地—充满江南气息的园林，叶园。长廊湖景，古雅庭院。这里淡雅而大气，是快节奏与民俗文化的交融，这里幽静而明快，是远道而来亦或小憩而居的别样天地。时生隅的新老朋友，我们在叶园，期待与您共襄当下景，把时光言欢。
2018年开业2018年装修143间房。云和夜泊酒店(上


In [5]:
vocab = sorted(set(text2))
print ('{} unique characters'.format(len(vocab)))

3014 unique characters


In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text2])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '%' :   4,
  '&' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  '+' :  10,
  ',' :  11,
  '-' :  12,
  '.' :  13,
  '/' :  14,
  '0' :  15,
  '1' :  16,
  '2' :  17,
  '3' :  18,
  '4' :  19,
  ...
}


In [8]:
# 设定每个输入句子长度的最大值
seq_length = 100
examples_per_epoch = len(text2)//seq_length

# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

2
0
1
9
年


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(25):
  print(repr(''.join(idx2char[item.numpy()])))

'2019年开业60间房。上海时生隅园林文化酒店：2017年的秋末，时生隅初见。彼时的心愿，想要在快节奏的都市，寻一处“讲诉时光和生命的角落”。不问对错，不闻浮沉。这便是“时生隅”的由来。 这一次，我们秉'
'承初心，在闹中取静的川沙地铁站商圈，觅得一方天地—充满江南气息的园林，叶园。长廊湖景，古雅庭院。这里淡雅而大气，是快节奏与民俗文化的交融，这里幽静而明快，是远道而来亦或小憩而居的别样天地。时生隅的新老朋'
'友，我们在叶园，期待与您共襄当下景，把时光言欢。\n2018年开业2018年装修143间房。云和夜泊酒店(上海浦东国际机场店)，位于浦东新区祝桥秋亭路，毗邻上海浦东国际机场，车程20分钟，距上海迪士尼18'
'分钟车程，距上海野生动物园21分钟车程。酒店地理位置优越，并配备大巴、中巴、豪华轿车接送客户。酒店拥有房间98间，大床30间，行政大床15间，标间30间，亲子房23间。\n2015年开业2017年装修23'
'6间房。维也纳酒店（上海浦东机场店）位于浦东新区川南奉公路，近晨阳路，与迪士尼直线距离约8公里，可便捷到达地铁2号线凌空路站，交通便利。酒店周围生活设施齐全，旅游资源众多，有上海新国际博览中心、迪士尼、'
'上海野生动物园、农艺大观园、三甲港滨海旅游区等。\u3000\u3000维也纳酒店（上海浦东机场店）是维也纳旗下的连锁酒店，装修豪华舒适，整体风格高贵典雅。客房宽敞明亮、温馨时尚，房内布置精美，处处体现人性化的理念。\u3000\u3000酒'
'店还有宽敞停车场、休闲茶吧、宽敞会议室等，同时还为您提供精品早餐、浦东机场接机（需预约）等服务，是商务、休闲、会务的理想酒店。\n2018年开业194间房。上海万信R酒店位于浦东新区崮山路，地处内环线陆家'
'嘴金融区内，近地铁6号线（北洋泾路站），地铁9号线（芳甸路站），配备专属停车场，自驾至新国际博览中心约10分钟，至外滩、世博园区、南京路步行街、上海自由贸易区、上海迪斯尼乐园约20分钟。\u3000\u3000\u3000\u3000酒店拥有'
'百余间崭新客房，精选配套，让舒适与轻奢相拥；酒店圣罗拉餐厅环境舒适安逸，主打西餐、辅以粤菜、川菜、沪菜，在此享受由港粤名厨精心烹饪的佳肴，舌尖不由自主的跳起了芭蕾。早餐的菜品琳琅满目可达40种，为保证口'
'味，餐品均选优质食材当天加工。\u300

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '2019年开业60间房。上海时生隅园林文化酒店：2017年的秋末，时生隅初见。彼时的心愿，想要在快节奏的都市，寻一处“讲诉时光和生命的角落”。不问对错，不闻浮沉。这便是“时生隅”的由来。 这一次，我们'
Target data: '019年开业60间房。上海时生隅园林文化酒店：2017年的秋末，时生隅初见。彼时的心愿，想要在快节奏的都市，寻一处“讲诉时光和生命的角落”。不问对错，不闻浮沉。这便是“时生隅”的由来。 这一次，我们秉'


In [12]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 17 ('2')
  expected output: 15 ('0')
Step    1
  input: 15 ('0')
  expected output: 16 ('1')
Step    2
  input: 16 ('1')
  expected output: 24 ('9')
Step    3
  input: 24 ('9')
  expected output: 938 ('年')
Step    4
  input: 938 ('年')
  expected output: 972 ('开')


In [13]:
# 批大小
BATCH_SIZE = 64

# 设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列，
# 所以它不会试图在内存中重新排列整个序列。相反，
# 它维持一个缓冲区，在缓冲区重新排列元素。） 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [14]:
# 词集的长度
vocab_size = len(vocab)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [15]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 3014) # (batch_size, sequence_length, vocab_size)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           771584    
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 3014)          3089350   
Total params: 7,799,238
Trainable params: 7,799,238
Non-trainable params: 0
_________________________________________________________________


In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 3014)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       8.011231


In [20]:
model.compile(optimizer='adam', loss=loss)
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [21]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 138 steps
Epoch 1/10
138/138 [==============================] - 267s 2s/step - loss: 5.3146
Epoch 2/10
138/138 [==============================] - 250s 2s/step - loss: 3.3793
Epoch 3/10
138/138 [==============================] - 239s 2s/step - loss: 2.8448
Epoch 4/10
138/138 [==============================] - 233s 2s/step - loss: 2.5451
Epoch 5/10
138/138 [==============================] - 233s 2s/step - loss: 2.3327
Epoch 6/10
138/138 [==============================] - 233s 2s/step - loss: 2.1595
Epoch 7/10
138/138 [==============================] - 491s 4s/step - loss: 2.0118
Epoch 8/10
138/138 [==============================] - 261s 2s/step - loss: 1.8756
Epoch 9/10
138/138 [==============================] - 249s 2s/step - loss: 1.7486
Epoch 10/10
138/138 [==============================] - 256s 2s/step - loss: 1.6311


In [22]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            771584    
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 3014)           3089350   
Total params: 7,799,238
Trainable params: 7,799,238
Non-trainable params: 0
_________________________________________________________________


In [25]:
def generate_text(model, start_string):
  # 评估步骤（用学习过的模型生成文本）

  # 要生成的字符个数
  num_generate = 1000

  # 将起始字符串转换为数字（向量化）
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 空字符串用于存储结果
  text_generated = []

  # 低温度会生成更可预测的文本
  # 较高温度会生成更令人惊讶的文本
  # 可以通过试验以找到最好的设定
  temperature = 1.0

  # 这里批大小为 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # 删除批次的维度
      predictions = tf.squeeze(predictions, 0)

      # 用分类分布预测模型返回的字符
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))
print(generate_text(model, start_string=u"靠近 "))

靠近 著名品牌“古玲玉兰””——炮上餐厅、国际房内等游玩客房，是美味锻炼的好去处。酒店设有站式餐厅，可以满足旅客的需求，更有高速的无线网络，特惠房间床，配备有洗衣架和4间，酒店客房均全网落地窗，透过不足门口。
2015年开业2013年装修33间房。99旅馆连锁(上海五代中心地铁站店)位于黄浦区打浦江新城路4号门斜对面靠近35路梅南路口，旁，地理罗山路之内，紧靠人民广场、南京路步行街等旅游景点，仅15分钟可到。酒店地处于上海松江团队带，周边围绕着名设计，营业丰富特色共商业氛围，更有近百安购物中心。　　海友客栈（上海徐家汇繁华客房，房内设有浴室独立空调、独立空调、电视、24小时热水等设施，无线网络全覆盖，为您营造安全、舒适、住宿的温暖。
2019年开业198间房。汉庭酒店（上海共平路店）处黄浦江中心，地处地铁四号线中山北路站南广场，交通极其便利，是尽外，驱车至闵行体育公园仅5分钟、三、好地玻璃窗、沐浴露和用品(p记))、高星级品牌旗下服务，酒店灵魂提倡人们需要及提供标准服务和免费停车问等，是您商务旅行之家的宾客商务.无论您与异体验探索机场。　　铂·无线wifi，独立冷暖空调、有烤线光纤上网、电量，还有高架机和开接待朋友酒店的处领名的上海旅游区域，是你在浦东休闲的住宿环境，流连忘返入住酒店，前台交通是您放松游休闲的亲切身处。
2018年开业57间房。89旅馆连锁（上海交大店）位于上海市宝山区沪太路水产村，毗邻金桥进口工业园区，东靠汽车南路，致力于为追求商务客人提供宽带上网，纵体的外滩风景区域；3分钟的车程即可从到达酒店；G60沪昆高速江北路2016等交通便捷。地理位置优越，交通便利。
2019年开业36间房。贝壳酒店(上海五角场店)坐落于环境优越的临近相连路店)毗邻七宝老街、上海迪士尼度假区东明公园、仁济医院。 酒店客房设计凸显《云翔裸温摄餐》还提供细致每一让回家的到来，您在爱揽这望自在这一靠宁静。
2008年开业2017年装修102间房。上海三甲港凯悦酒店位于黄浦区长江南路（陆盛路口往人民广场步行街涞科路淀山西路，西藏北路地铁站即可前往。
2012年开业2016年装修142间房。汉庭酒店（上海火车站店）位于中国北路，距上海火车站可乘坐15号线至国家会展中心约40分钟，到上海著名的“七宝”街仅需25分钟车程。酒店上海素重装修为华住宗旨，倾注为「神咖啡」，清爽真溢艺术“A级